In [ ]:
import os
concat_path = "XTT22_train.fa"

In [2]:
full_fasta_path = os.path.abspath(concat_path)
output_dir = os.path.abspath("preprocessed_data")
output_yaml = f"""
- datapaths: ["{full_fasta_path}"]
  output_dir: "{output_dir}"
  output_prefix: XTT22_train
  train_split: 0.9
  valid_split: 0.05
  test_split: 0.05
  overwrite: True
  embed_reverse_complement: true
  random_reverse_complement: 0.0
  random_lineage_dropout: 0.0
  include_sequence_id: false
  transcribe: "back_transcribe"
  force_uppercase: false
  indexed_dataset_dtype: "uint8"
  tokenizer_type: "Byte-Level"
  vocab_file: null
  vocab_size: null
  merges_file: null
  pretrained_tokenizer_model: null
  special_tokens: null
  fast_hf_tokenizer: true
  append_eod: true
  enforce_sample_length: null
  ftfy: false
  workers: 1
  preproc_concurrency: 100000
  chunksize: 25
  drop_empty_sequences: true
  nnn_filter: false  # If you split your fasta on NNN (in human these are contigs), then you should set this to true.
  seed: 12342  # Not relevant because we are not using random reverse complement or lineage dropout.
"""
with open("preprocess_config.yaml", "w") as f:
    print(output_yaml, file=f)

In [3]:
!preprocess_evo2 --config preprocess_config.yaml

[NeMo I 2025-05-24 12:37:06 nemo_logging:393] Using byte-level tokenization
[NeMo I 2025-05-24 12:37:06 nemo_logging:393] Created temporary binary datasets: /workspace/preprocessed_data/XTT22_train_byte-level_train.bin.tmp /workspace/preprocessed_data/XTT22_train_byte-level_val.bin.tmp /workspace/preprocessed_data/XTT22_train_byte-level_test.bin.tmp
[NeMo I 2025-05-24 13:12:11 nemo_logging:393] Average preprocessing time per sequence: 0.04470627161196968
[NeMo I 2025-05-24 13:12:11 nemo_logging:393] Average indexing time per sequence: 0.1463382052460373
[NeMo I 2025-05-24 13:12:11 nemo_logging:393] Number of sequences processed: 12092
[NeMo I 2025-05-24 13:12:11 nemo_logging:393] Finished preprocessing XTT22_train ([PosixPath('/workspace/XTT22_train.fa')]) in 2105.082 seconds with 1 workers.


In [4]:
!ls -lh preprocessed_data/

total 14G
-rw-r--r-- 1 root root 936M May 24 13:11 XTT22_train_byte-level_test.bin
-rw-r--r-- 1 root root  12K May 24 13:12 XTT22_train_byte-level_test.idx
-rw-r--r-- 1 root root  13G May 24 13:12 XTT22_train_byte-level_train.bin
-rw-r--r-- 1 root root 213K May 24 13:12 XTT22_train_byte-level_train.idx
-rw-r--r-- 1 root root 411M May 24 13:12 XTT22_train_byte-level_val.bin
-rw-r--r-- 1 root root  12K May 24 13:12 XTT22_train_byte-level_val.idx


In [10]:
!evo2_convert_to_nemo2 \
  --model-path /workspace/savanna_evo2_1b_base/savanna_evo2_1b_base.pt \
  --model-size 1b --output-dir nemo2_evo2_1b_8k

Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda129.so')
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
[NeMo I 2025-05-24 15:02:39 nemo_logging:393] Using byte-level tokenization
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo W 2025-05-24 15:02:39 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
    
[NeMo I 2025-05-24 15:02:39 nemo_logging:393] Fixing mis-match between ddp-config & mcore-optimizer config
[NeMo I 2025-05-24 15:02:39 nemo_logging:393] Rank 0 has data parallel group : [0]
[NeMo I 2025-05-24 15:02:39 nemo_logging:393] Rank 0 has combined group of data parallel and context parallel : [0

In [11]:
import os
from pathlib import Path
output_pfx = str(Path(os.path.abspath("preprocessed_data"))/"XTT22_train_byte-level")
output_yaml = f"""
- dataset_prefix: {output_pfx}_train
  dataset_split: train
  dataset_weight: 1.0
- dataset_prefix: {output_pfx}_val
  dataset_split: validation
  dataset_weight: 1.0
- dataset_prefix: {output_pfx}_test
  dataset_split: test
  dataset_weight: 1.0
"""
with open("training_data_config.yaml", "w") as f:
    print(output_yaml, file=f)

In [12]:
!train_evo2 \
    -d training_data_config.yaml \
    --dataset-dir {preprocessed_data} \
    --model-size 1b \
    --devices 1 \
    --num-nodes 1 \
    --seq-length 1 \
    --micro-batch-size 1 \
    --lr 0.0001 \
    --warmup-steps 5 \
    --max-steps 100 \
    --ckpt-dir nemo2_evo2_1b_8k \
    --clip-grad 1 \
    --wd 0.01 \
    --activation-checkpoint-recompute-num-layers 1 \
    --val-check-interval 50 \
    --ckpt-async-save

Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda129.so')
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
[NeMo I 2025-05-24 15:03:17 nemo_logging:393] Using byte-level tokenization
[NeMo W 2025-05-24 15:03:17 nemo_logging:405] WandB is currently turned off.
[NeMo W 2025-05-24 15:03:17 nemo_logging:405] User-set tensorboard is currently turned off. Internally one may still be set by NeMo2.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo I 2025-05-24 15:03:17 nemo_logging:393] Experiments will be logged at results/evo2/dev
[NeMo W 2025-05-24 15:03:17 

In [14]:
!pip install -q peft

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/opt_einsum-3.4.0-py3.12.egg is deprecated. pip 25.1 will enf

In [20]:
!cp /workspace/bionemo_train.py /usr/local/lib/python3.12/dist-packages/bionemo/evo2/run/train.py

In [ ]:

!train_evo2 \
    -d training_data_config.yaml \
    --dataset-dir {preprocessed_data} \
    --model-size 1b \
    --devices 1 \
    --num-nodes 1 \
    --seq-length 1 \
    --micro-batch-size 1 \
    --lr 0.0001 \
    --warmup-steps 5 \
    --max-steps 300000 \
    --ckpt-dir nemo2_evo2_1b_8k \
    --clip-grad 1 \
    --wd 0.01 \
    --activation-checkpoint-recompute-num-layers 1 \
    --val-check-interval 1000 \
    --ckpt-async-save

Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda129.so')
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
[NeMo I 2025-06-01 15:37:07 nemo_logging:393] Using byte-level tokenization
[NeMo W 2025-06-01 15:37:07 nemo_logging:405] WandB is currently turned off.
[NeMo W 2025-06-01 15:37:07 nemo_logging:405] User-set tensorboard is currently turned off. Internally one may still be set by NeMo2.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo I 2025-06-01 15:37:07 nemo_logging:393] Experiments will be logged at results/evo2/dev
[NeMo W 2025-06-01 15:37:07 

In [ ]:
import os
import stat

# 检查文件路径
file_path = "/usr/local/bin/train_evo2"

# 首先检查文件是否存在
if os.path.exists(file_path):
    print(f"文件存在: {file_path}")
    
    # 获取文件信息
    file_stat = os.stat(file_path)
    print(f"文件大小: {file_stat.st_size} 字节")
    print(f"文件权限: {stat.filemode(file_stat.st_mode)}")
    print(f"是否可执行: {os.access(file_path, os.X_OK)}")
    
    # 检查文件类型
    with open(file_path, 'rb') as f:
        first_bytes = f.read(100)
        print(f"文件开头字节: {first_bytes[:50]}")
        
        # 检查是否是文本文件
        try:
            first_text = first_bytes.decode('utf-8')
            print("这是一个文本文件")
            print(f"文件开头内容: {first_text[:100]}...")
        except UnicodeDecodeError:
            print("这是一个二进制文件")
    
    # 如果是Python脚本，显示基本信息（不显示完整内容）
    if file_path.endswith('.py') or 'python' in first_text.lower():
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            print(f"\n总行数: {len(lines)}")
            print("文件头部信息（前5行）:")
            for i, line in enumerate(lines[:100]):
                print(f"{i+1}: {line.rstrip()}")
else:
    print(f"文件不存在: {file_path}")
    
    # 检查可能的替代路径
    possible_paths = [
        "/usr/local/bin/train_evo2",
        "/usr/local/lib/python3.12/dist-packages/bionemo/evo2/run/train.py",
        "/usr/local/lib/python3.12/dist-packages/bionemo/evo2/train.py"
    ]
    
    print("\n检查其他可能的路径:")
    for path in possible_paths:
        if os.path.exists(path):
            print(f"✓ 找到: {path}")
        else:
            print(f"✗ 不存在: {path}")

In [5]:
#!/usr/bin/env python3
"""
打印 BioNeMo Evo2 训练脚本内容
"""

import os
from pathlib import Path

def print_file_content():
    """打印指定文件的内容"""
    
    file_path = "/usr/local/lib/python3.12/dist-packages/bionemo/evo2/run/train.py"
    
    print("=" * 80)
    print(f"文件路径: {file_path}")
    print("=" * 80)
    
    try:
        # 检查文件是否存在
        if not os.path.exists(file_path):
            print(f"❌ 文件不存在: {file_path}")
            
            # 尝试查找类似的文件
            print("\n正在搜索相关文件...")
            base_dir = "/usr/local/lib/python3.12/dist-packages/"
            
            if os.path.exists(base_dir):
                print(f"✓ 基础目录存在: {base_dir}")
                
                # 搜索 bionemo 相关目录
                for root, dirs, files in os.walk(base_dir):
                    if "bionemo" in root.lower():
                        print(f"找到相关目录: {root}")
                        if "train.py" in files:
                            print(f"  -> 包含 train.py: {os.path.join(root, 'train.py')}")
            else:
                print(f"❌ 基础目录不存在: {base_dir}")
            
            return False
        
        # 读取并打印文件内容
        print(f"✓ 文件存在，正在读取内容...\n")
        
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # 打印文件信息
        lines = content.split('\n')
        print(f"文件大小: {len(content)} 字符")
        print(f"行数: {len(lines)}")
        print("-" * 80)
        
        # 打印内容（带行号）
        for i, line in enumerate(lines, 1):
            print(f"{i:4d}: {line}")
        
        print("-" * 80)
        print("✓ 文件内容已打印完成")
        return True
        
    except PermissionError:
        print(f"❌ 权限不足，无法读取文件: {file_path}")
        print("请尝试使用 sudo 运行此脚本")
        return False
        
    except Exception as e:
        print(f"❌ 读取文件时发生错误: {e}")
        return False

def search_alternative_paths():
    """搜索可能的替代路径"""
    
    potential_paths = [
        "/usr/local/lib/python3.12/dist-packages/bionemo/evo2/run/train.py",
        "/usr/local/lib/python3.11/dist-packages/bionemo/evo2/run/train.py",
        "/usr/lib/python3.12/dist-packages/bionemo/evo2/run/train.py",
        "/usr/lib/python3.11/dist-packages/bionemo/evo2/run/train.py",
    ]
    
    # 也检查当前用户的site-packages
    import site
    user_site = site.getusersitepackages()
    if user_site:
        potential_paths.append(f"{user_site}/bionemo/evo2/run/train.py")
    
    print("\n搜索可能的路径:")
    print("-" * 50)
    
    found_files = []
    for path in potential_paths:
        if os.path.exists(path):
            print(f"✓ 找到: {path}")
            found_files.append(path)
        else:
            print(f"✗ 不存在: {path}")
    
    return found_files

def main():
    """主函数"""
    print("BioNeMo Evo2 训练脚本内容查看器")
    print("=" * 80)
    
    # 首先尝试打印目标文件
    success = print_file_content()
    
    if not success:
        # 如果失败，搜索替代路径
        found_files = search_alternative_paths()
        
        if found_files:
            print(f"\n找到 {len(found_files)} 个相关文件。")
            for i, file_path in enumerate(found_files, 1):
                print(f"\n{i}. 正在打印: {file_path}")
                print("=" * 80)
                
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                    
                    lines = content.split('\n')
                    print(f"文件大小: {len(content)} 字符")
                    print(f"行数: {len(lines)}")
                    print("-" * 80)
                    
                    for line_num, line in enumerate(lines, 1):
                        print(f"{line_num:4d}: {line}")
                    
                    print("-" * 80)
                    
                except Exception as e:
                    print(f"❌ 读取文件 {file_path} 时发生错误: {e}")
        else:
            print("\n❌ 未找到任何相关的训练脚本文件")

if __name__ == "__main__":
    main() 

BioNeMo Evo2 训练脚本内容查看器
文件路径: /usr/local/lib/python3.12/dist-packages/bionemo/evo2/run/train.py
✓ 文件存在，正在读取内容...

文件大小: 29121 字符
行数: 717
--------------------------------------------------------------------------------
   1: # SPDX-FileCopyrightText: Copyright (c) 2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
   2: # SPDX-FileCopyrightText: Copyright (c) 2024 Arc Institute. All rights reserved.
   3: # SPDX-FileCopyrightText: Copyright (c) 2024 Michael Poli. All rights reserved.
   4: # SPDX-FileCopyrightText: Copyright (c) 2024 Stanford University. All rights reserved
   5: # SPDX-License-Identifier: LicenseRef-Apache2
   6: #
   7: # Licensed under the Apache License, Version 2.0 (the "License");
   8: # you may not use this file except in compliance with the License.
   9: # You may obtain a copy of the License at
  10: #
  11: #     http://www.apache.org/licenses/LICENSE-2.0
  12: #
  13: # Unless required by applicable law or agreed to in writing, software
  14: # distr

In [4]:
!evo2_convert_to_nemo2 \
  --model-path /workspace/savanna_evo2_7b/savanna_evo2_7b.pt \
  --model-size 7b --output-dir nemo2_evo2_7b

^C
Traceback (most recent call last):
  File "/usr/local/bin/evo2_convert_to_nemo2", line 4, in <module>
    from bionemo.evo2.utils.checkpoint.convert_to_nemo import main
  File "/usr/local/lib/python3.12/dist-packages/bionemo/evo2/utils/checkpoint/convert_to_nemo.py", line 24, in <module>
    from nemo.collections.llm.gpt.model.hyena import (
  File "/usr/local/lib/python3.12/dist-packages/nemo/collections/llm/__init__.py", line 16, in <module>
    from nemo.utils.import_utils import safe_import
  File "/usr/local/lib/python3.12/dist-packages/nemo/utils/__init__.py", line 17, in <module>
    from nemo.utils.cast_utils import (
  File "/usr/local/lib/python3.12/dist-packages/nemo/utils/cast_utils.py", line 17, in <module>
    import torch
  File "/usr/local/lib/python3.12/dist-packages/torch/__init__.py", line 411, in <module>
    from torch._C import *  # noqa: F403
    ^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 463, in _lock_unlock_module
KeyboardInterrupt


In [ ]:
!train_evo2 \
    -d training_data_config.yaml \
    --dataset-dir {preprocessed_data} \
    --model-size 7b \
    --devices 4 \
    --num-nodes 1 \
    --seq-length 1 \
    --micro-batch-size 1 \
    --lr 0.0001 \
    --warmup-steps 5 \
    --max-steps 200000 \
    --ckpt-dir nemo2_evo2_7b \
    --clip-grad 1 \
    --wd 0.01 \
    --activation-checkpoint-recompute-num-layers 1 \
    --val-check-interval 1000 \
    --ckpt-async-save

In [1]:
import os
import tempfile

os.environ['CUDA_VISIBLE_DEVICES'] = '4'
os.environ['TORCHINDUCTOR_CACHE_DIR'] = tempfile.mkdtemp()
os.environ['USER'] = 'user'
os.environ['USERNAME'] = 'user'
!predict_evo2 \
    --fasta sequence_truncation/biological_truncated_sequences_fixed.fasta \
    --ckpt-dir results/evo2/checkpoints/evo2--val_loss=1.4537-epoch=0-consumed_samples=100.0-last \
    --model-size 1b \
    --tensor-parallel-size 1 \
    --pipeline-model-parallel-size 1 \
    --context-parallel-size 1 \
    --batch-size 1 \
    --output-dir prediction_results_06082149 \
    --ckpt-format torch_dist \
    --output-log-prob-seqs \
    --log-prob-collapse-option mean \
    --prepend-bos

Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda129.so')
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo W 2025-06-08 13:50:10 nemo_logging:405] No version folders would be created under the log folder as 'resume_if_exists' is enabled.
[NeMo I 2025-06-08 13:50:10 nemo_logging:393] Experiments will be logged at /tmp/tmpb5yk13vp/default
[NeMo W 2025-06-08 13:50:10 nemo_logging:405] "update_logger_directory" is True. Overwriting tensorboard logger "save_dir" to /tmp/tmpb5yk13vp
[NeMo I 2025-06-08 13:50:10 nemo_logging:393] Using byte-level tokenization
[NeMo I 2025-06-08 13:50:11 nemo_logging:393] Rank 0 has data parallel group : [0]
[NeMo I 2025-06-08 13:50:11 nemo_logging:393] Rank 0 h

In [ ]:
import os
import tempfile

os.environ['CUDA_VISIBLE_DEVICES'] = '4'
os.environ['TORCHINDUCTOR_CACHE_DIR'] = tempfile.mkdtemp()
os.environ['USER'] = 'user'
os.environ['USERNAME'] = 'user'
!predict_evo2 \
    --fasta sequence_truncation/biological_truncated_sequences_fixed.fasta \
    --ckpt-dir nemo2_evo2_1b_8k \
    --model-size 1b \
    --tensor-parallel-size 1 \
    --pipeline-model-parallel-size 1 \
    --context-parallel-size 1 \
    --batch-size 1 \
    --output-dir prediction_results_nolog_XTT22_0shot \
    --ckpt-format torch_dist \
    --log-prob-collapse-option mean \
    --prepend-bos

Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda129.so')
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo W 2025-06-23 12:33:04 nemo_logging:405] No version folders would be created under the log folder as 'resume_if_exists' is enabled.
[NeMo I 2025-06-23 12:33:04 nemo_logging:393] Experiments will be logged at /tmp/tmp24sf05a5/default
[NeMo W 2025-06-23 12:33:04 nemo_logging:405] "update_logger_directory" is True. Overwriting tensorboard logger "save_dir" to /tmp/tmp24sf05a5
[NeMo I 2025-06-23 12:33:04 nemo_logging:393] Using byte-level tokenization
[NeMo I 2025-06-23 12:33:05 nemo_logging:393] Rank 0 has data parallel group : [0]
[NeMo I 2025-06-23 12:33:05 nemo_logging:393] Rank 0 h

IOStream.flush timed out
IOStream.flush timed out
